# Using Optimization Techniques to Generate Winning Lineups
___
Until I build one from scratch I'm going to be testing out `pydfs-lineup-optimizer` to generate optimal lineups with projections that I already have. More information about `pydfs-lineup-optimizer` can be found [here](https://github.com/DimaKudosh/pydfs-lineup-optimizer/tree/master/pydfs_lineup_optimizer/sites/fanduel).

**Here is one quick example of generating lineups based on average points per game.**

In [14]:
import pandas as pd
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, CSVLineupExporter

In [15]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('player-lists/FanDuel-NFL-2018-11-11-29736-players-list.csv')
for lineup in optimizer.optimize(n=10, max_exposure=0.3):
#     print(lineup)
    print(lineup.players)
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs)

[Ryan Fitzpatrick QB (TB), T.J. Yeldon RB (JAC), Melvin Gordon III RB (LAC), Tyreek Hill WR (KC), Cooper Kupp WR (LAR), DeSean Jackson WR (TB), Eric Ebron TE (IND), Seattle Seahawks D (SEA), Alvin Kamara RB (NO)]
154.546
60000.0
[Ryan Fitzpatrick QB (TB), T.J. Yeldon RB (JAC), Melvin Gordon III RB (LAC), Tyreek Hill WR (KC), Cooper Kupp WR (LAR), DeSean Jackson WR (TB), Eric Ebron TE (IND), Arizona Cardinals D (ARI), Alvin Kamara RB (NO)]
154.296
59400.0
[Ryan Fitzpatrick QB (TB), T.J. Yeldon RB (JAC), Melvin Gordon III RB (LAC), Tyreek Hill WR (KC), Calvin Ridley WR (ATL), Cooper Kupp WR (LAR), Eric Ebron TE (IND), Arizona Cardinals D (ARI), Alvin Kamara RB (NO)]
154.071
59700.0
[Matt Ryan QB (ATL), Todd Gurley II RB (LAR), Isaiah Crowell RB (NYJ), Brandon LaFell WR (OAK), DeSean Jackson WR (TB), Michael Thomas WR (NO), Jared Cook TE (OAK), Arizona Cardinals D (ARI), James White RB (NE)]
140.0
60000.0
[Matt Ryan QB (ATL), Todd Gurley II RB (LAR), Isaiah Crowell RB (NYJ), Brandon LaFel

**Now instead of just using the average points per game, we'll bring in our other projections that *should* be more accurate.**

In [16]:
qbs = pd.read_csv('projections/Week-10/DFS - Fanduel QB.csv')
rbs = pd.read_csv('projections/Week-10/DFS - Fanduel RB.csv')
wrs = pd.read_csv('projections/Week-10/DFS - Fanduel WR.csv')
tes = pd.read_csv('projections/Week-10/DFS - Fanduel TE.csv')
dst = pd.read_csv('projections/Week-10/DFS - Fanduel DST.csv')

In [17]:
qbs.head()

,Player,Opp,Pts,Salary,Pts Per 1k
0,Aaron Rodgers,vs. MIA,28.4,"8,600",3.3
1,Drew Brees,at CIN,25.3,"8,400",3.0
2,Patrick Mahomes,vs. ARI,29.2,"9,800",3.0
3,Jared Goff,vs. SEA,23.7,"8,100",2.9
4,Ryan Fitzpatrick,vs. WAS,21.3,"7,600",2.8


In [18]:
total_projections = pd.concat([qbs,rbs,wrs,tes,dst])
total_projections.rename(columns={"Player": "Nickname"}, inplace=True)

In [19]:
player_list=pd.read_csv('player-lists/FanDuel-NFL-2018-11-11-29736-players-list.csv')
player_list.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,29736-30447,RB,Todd,Todd Gurley II,Gurley II,26.833333,9,10800,SEA@LAR,LAR,SEA,NaN,NaN,NaN,NaN,NaN
1,29736-57439,QB,Patrick,Patrick Mahomes,Mahomes,27.860001,9,9800,ARI@KC,KC,ARI,NaN,NaN,NaN,NaN,NaN
2,29736-45229,RB,Kareem,Kareem Hunt,Hunt,20.888889,9,9000,ARI@KC,KC,ARI,NaN,NaN,NaN,NaN,NaN


In [20]:
player_list.columns

Index(['Id', 'Position', 'First Name', 'Nickname', 'Last Name', 'FPPG',
       'Played', 'Salary', 'Game', 'Team', 'Opponent', 'Injury Indicator',
       'Injury Details', 'Tier', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')

> Merging those 2 and making the swap:

In [21]:
master_projections = pd.merge(left=player_list, right=total_projections, how='inner', on='Nickname', suffixes=['_FD', '_ballers'])
master_projections['FPPG'] = master_projections['Pts Per 1k'] # Experiment with Pts OR Pts Per 1k
master_projections.drop(['Opp', 'Pts', 'Salary_ballers', 'Pts Per 1k'],axis=1, inplace=True)
master_projections.rename(columns={'Salary_FD':'Salary'}, inplace=True)
master_projections.to_csv('projections/Week-10/master_projections.csv')

In [22]:
master_projections.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,29736-57439,QB,Patrick,Patrick Mahomes,Mahomes,3.0,9,9800,ARI@KC,KC,ARI,NaN,NaN,NaN,NaN,NaN
1,29736-45229,RB,Kareem,Kareem Hunt,Hunt,3.5,9,9000,ARI@KC,KC,ARI,NaN,NaN,NaN,NaN,NaN
2,29736-42104,RB,Alvin,Alvin Kamara,Kamara,2.8,8,8800,NO@CIN,NO,CIN,NaN,NaN,NaN,NaN,NaN


___
# Alright! Now let's just use our optimizer tools from above to generate some better lineups

In [23]:
low_tes = list(master_projections.loc[(master_projections['FPPG'] <= 0.80) & (master_projections['Position']=='TE'), :]['Nickname'].values)
out_players = list(master_projections.loc[(master_projections['Injury Indicator'] == 'IR') | (master_projections['Injury Indicator'] == 'O'), :]['Nickname'].values)
low_scorers = list(master_projections.loc[master_projections['FPPG'] < 0.80, :]['Nickname'].values)

fade_players = ['Rob Gronkowski', 'Matthew Stafford', 'Alex Smith', 'Joe Mixon', 'Kerryon Johnson', 'Adrian Peterson', 'Allen Robinson', 'Corey Davis', 'T.Y. Hilton']
# wrs_bad_matchups

all_removals = low_tes + out_players + low_scorers + fade_players
all_removals = set(all_removals)

In [24]:
'Mike Gesicki' in all_removals

True

In [36]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-10/master_projections.csv')

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'WR'])
# optimizer.set_positions_for_same_team(['RB', 'D'])

# Making more randomness
optimizer.set_deviation(0.06, 0.18)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 12 lineups
for lineup in optimizer.optimize(n=10, max_exposure=.13):
    print(lineup, '\n')

 1. QB    Jared Goff                    QB    LAR            2.9     8100.0$   
 2. RB    Nick Chubb                    RB    CLE            2.6     6700.0$   
 3. RB    Aaron Jones                   RB    GB             2.5     6500.0$   
 4. WR    Cooper Kupp                   WR    LAR            2.4     6500.0$   
 5. WR    Jarvis Landry                 WR    CLE            2.4     6400.0$   
 6. WR    Michael Thomas                WR    NO             3.6     8600.0$   
 7. TE    O.J. Howard                   TE    TB             2.3     6500.0$   
 8. D     New York Jets                 D     NYJ            4.4     4900.0$   
 9. FLEX  Kenny Golladay                WR    DET            2.1     5800.0$   

Fantasy Points 25.2
Salary 60000.0 

 1. QB    Drew Brees                    QB    NO             3.0     8400.0$   
 2. RB    Nick Chubb                    RB    CLE            2.6     6700.0$   
 3. RB    Aaron Jones                   RB    GB             2.5     6500.0$   
 4

In [37]:
exporter = CSVLineupExporter(optimizer.optimize(n=10, max_exposure= .13))
exporter.export('submissions/Week-10/fanduel-lineups.csv')

In [38]:
# Doing the second half with a RB + D stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-10/master_projections.csv')

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['RB', 'D'])
# optimizer.set_positions_for_same_team(['RB', 'D'])

# Making more randomness
optimizer.set_deviation(0.06, 0.18)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 12 lineups
for lineup in optimizer.optimize(n=10, max_exposure=.13):
    print(lineup, '\n')

 1. QB    Ryan Fitzpatrick              QB    TB             2.8     7600.0$   
 2. RB    Nick Chubb                    RB    CLE            2.6     6700.0$   
 3. RB    Aaron Jones                   RB    GB             2.5     6500.0$   
 4. WR    Jarvis Landry                 WR    CLE            2.4     6400.0$   
 5. WR    Michael Thomas                WR    NO             3.6     8600.0$   
 6. WR    Kenny Golladay                WR    DET            2.1     5800.0$   
 7. TE    Travis Kelce                  TE    KC             3.0     8000.0$   
 8. D     New York Jets                 D     NYJ            4.4     4900.0$   
 9. FLEX  Isaiah Crowell                RB    NYJ            1.7     5400.0$   

Fantasy Points 25.1
Salary 59900.0 

 1. QB    Aaron Rodgers                 QB    GB             3.3     8600.0$   
 2. RB    Nick Chubb                    RB    CLE            2.6     6700.0$   
 3. RB    Isaiah Crowell                RB    NYJ            1.7     5400.0$   
 4

In [39]:
exporter = CSVLineupExporter(optimizer.optimize(n=10, max_exposure= .13))
exporter.export('submissions/Week-10/fanduel-lineups_2.csv')